<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Лабораторная работа 7.</b> Обработка данных с использованием PIG, Spark SQL, HIVE</div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin.study@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Обработка данных с PIG</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#1a">Запуск приложения на Cloudera</a></li>
                <li><a href="#1b">Запуск приложения в AWS</a></li>
                <li><a href="#1b">Запуск приложения в Azure</a></li>
            </ol>
        </li>
        <li><a href="#2">Обработка данных с Spark SQL и DataFrame</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#2a">Запуск приложения на Cloudera</a></li>
                <li><a href="#2b">Запуск приложения в AWS</a></li>
                <li><a href="#3c">Запуск приложения в Azure</a></li>
            </ol>
        </li>
        <li><a href="#3">Обработка данных с Hive</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#3a">Запуск приложения на Cloudera</a></li>
                <li><a href="#3b">Запуск приложения в AWS</a></li>
                <li><a href="#3c">Запуск приложения в Azure</a></li>
            </ol>
        </li>
        <li><a href="#4">Источники</a>
        </li>
    </ol>
</div>

In [1]:
from IPython.core.display import HTML
HTML("<link rel='stylesheet' href='css/style.css'>")

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Обработка данных с PIG</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "1a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Запуск приложения на Cloudera
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#1b">Далее</a>
            </div>
        </div>
    </div>
</div>

<p>Исходный файл с данными</p>
<p><a href="https://github.com/alanfgates/programmingpig/blob/master/data/NYSE_dividends">https://github.com/alanfgates/programmingpig/blob/master/data/NYSE_dividends</a></p>

</p>Скопируйте локальный файл <span class="code-font">NYSE_dividends.txt</span> в <span class="code-font code-key">HDFS</span></p>

In [ ]:
hdfs dfs -copyFromLocal /home/cloudera/NYSE_dividends.txt  /tmp/NYSE_dividends.txt

<p>Проверьте, что файл успешно скопирован</p>

In [ ]:
hdfs dfs -ls /tmp

<p>Запустите интерактивный режим <span class="code-font code-key">Pig</span> с использованием <span class="code-font code-key">Grunt shell</span></p>

In [ ]:
pig -x mapreduce

<p>Запустите следующий код в интерактивной среде</p>

In [ ]:
#Формируем схему разбора текстового файла
divs = LOAD '/tmp/NYSE_dividends.txt' AS (exchange:chararray, symbol:chararray, date:chararray, dividends:float);

#Для вывода только 5 строк
#Первое. Ограничиваем
divs_5 = LIMIT divs 5;

#Второе. Выводим на экран 
DUMP divs_5

#Cхема trimmed
DESCRIBE divs

<div class="msg-block msg-info">
  <span class="msg-text-info">Замечание! </span> <p>Основные команды в Pig можно найти по следующей ссылке:</p>
  <p><a href="http://mortar-public-site-content.s3-website-us-east-1.amazonaws.com/">http://mortar-public-site-content.s3-website-us-east-1.amazonaws.com/</a></p>
</div>

<div class="msg-block msg-warning">
  <span class="msg-text-warn">Предупреждение! </span>
  <p>Если не запускается, то попробуйте команду для выхода HDFS из безопасного режима</p>
      <div class="code-block"><p class="code-font"><span class="code-key">hdfs</span> dfsadmin -safemode leave</p></div>
</div>

<p>Далее выполните следующие команды в интерактивной среде</p>

In [ ]:
#Для каждой строки оставляем данные из столбцов symbol и dividents
trimmed = FOREACH divs GENERATE symbol, dividends;

#Выводим 5 строк
trimmed_5 = LIMIT trimmed 5;
DUMP trimmed_5;

#Cхема trimmed
DESCRIBE trimmed_5

<p>Запустите следующее</p>

In [ ]:
#Группируем по symbol
grpd = GROUP trimmed BY symbol;

#Выводим 5 строк
grpd_5 = LIMIT grpd 5;
DUMP grpd_5;

#Cхема trimmed
DESCRIBE grpd

<p>Далее</p>

In [ ]:
#Для каждой группы определяем среднее значение дивидентов
avgdiv = FOREACH grpd GENERATE group, AVG(trimmed.dividends);

#Выводим 5 строк
avgdiv_5 = LIMIT avgdiv 5;
DUMP avgdiv_5

#Cхема avgdiv
DESCRIBE avgdiv

<p>Сохраните данные <span class="code-font">avgdiv</span> в <span class="code-font code-key">HDFS</span></p>

In [ ]:
STORE avgdiv INTO '/tmp/pigOut';

<div class="msg-block msg-warning">
  <span class="msg-text-warn">Предупреждение! </span>
  <p>Если директория pigOut уже существует, то перед запуском удалите её</p>
      <div class="code-block"><p class="code-font"><span class="code-key">hdfs</span> dfs -rm -r /tmp/pigOut</p></div>
</div>

<p>Проверьте, что файлы успешно созданы</p>

In [ ]:
hdfs dfs -ls /tmp/pigOut

<p>Проверьте содержание <span class="code-font">part</span>-файлов</p>

In [ ]:
hdfs dfs -cat /tmp/pigOut/part-r-00000

<p>Отобразите логический, физический и MapReduce планы выполнения</p>

In [ ]:
EXPLAIN avgdiv 

<p>Отобразите выборочные данные в табличном виде со схемами</p>

In [ ]:
ILLUSTRATE avgdiv

<p>Запустите файл с кодом</p>

In [ ]:
#Полный путь к файлу, в котором будет храниться код программы
pig_file = "YOUR_PATH/myPig.pig"

<p>Записываем код в файл myPig.pig</p>

In [ ]:
%%writefile $pig_file

divs = LOAD '/tmp/NYSE_dividends.txt' AS (exchange:chararray, symbol:chararray, date:chararray, dividends:float);
trimmed = FOREACH divs GENERATE symbol, dividends;
grpd = GROUP trimmed BY symbol;
avgdiv = FOREACH grpd GENERATE group, AVG(trimmed.dividends);
STORE avgdiv INTO '/tmp/pigOut2';

<div class="msg-block msg-warning">
  <span class="msg-text-warn">Предупреждение! </span>
  <p>Если директория pigOut2 уже существует, то перед запуском удалите её</p>
      <div class="code-block"><p class="code-font"><span class="code-key">hdfs</span> dfs -rm -r /tmp/pigOut2</p></div>
</div>

In [ ]:
pig -x mapreduce YOUR_PATH/myPig.pig

<p>Проверьте, что файлы успешно созданы</p>

In [ ]:
hdfs dfs -ls /tmp/pigOut2

<p>Проверьте содержание <span class="code-font">part</span>-файлов</p>

In [ ]:
 hdfs dfs -cat /tmp/pigOut2/part-r-00000

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Обработка данных с Spark SQL и DataFrame</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<div class="msg-block msg-info">
  <span class="msg-text-info">Замечание! </span> <p>Код выполнить в терминале или Jupyter</p>
</div>

<a name = "1a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Запуск приложения на Cloudera
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#1b">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql import Row

In [ ]:
sqlContext = SQLContext(sc)

In [ ]:
lines = sc.textFile("/tmp/NYSE_dividends.txt")

In [ ]:
lines.take(5)

In [ ]:
parts = lines.map(lambda line: line.split("\t"))

In [ ]:
parts.take(5)

In [ ]:
rowDivs = parts.map(lambda p: Row(exchange=p[0], symbol=p[1], date=p[2], dividends=float(p[3])))

In [ ]:
rowDivs.take(5)

In [ ]:
dfDivs = sqlContext.createDataFrame(rowDivs)

In [ ]:
dfDivs.show(5)

In [ ]:
dfTrimmed = dfDivs.select("symbol", "dividends")

dfTrimmed.show(5)

In [ ]:
grpd = dfTrimmed.groupBy("symbol")

In [ ]:
dfAvgdiv = grpd.avg()

dfAvgdiv.show(5)

In [ ]:
dfAvgdiv = dfDivs.select("symbol", "dividends") \
                  .groupBy("symbol") \
                  .avg()

dfAvgdiv.show(5)

In [ ]:
sqlContext.registerDataFrameAsTable(dfDivs, "divs")

dfAvgdiv = sqlContext.sql("SELECT symbol, AVG(dividends) FROM divs GROUP BY symbol")

dfAvgdiv.show(5)

In [ ]:
rddAvgdiv = dfAvgdiv.rdd

In [ ]:
def convertToString(row):
    return row[0] + " " + str(row[1])

rddAvgdivStr = rddAvgdiv.map(convertToString)

In [ ]:
rddAvgdivStr.getNumPartitions()

In [ ]:
rddAvgdivStr_c = rddAvgdivStr.coalesce(1)

rddAvgdivStr_c.saveAsTextFile("/tmp/sparkSQLOut")

<div class="msg-block msg-warning">
  <span class="msg-text-warn">Предупреждение! </span>
  <p>Если директория sparkSQLOut уже существует, то перед запуском удалите её</p>
      <div class="code-block"><p class="code-font"><span class="code-key">hdfs</span> dfs -rm -r /tmp/sparkSQLOut</p></div>
</div>

In [ ]:
hdfs dfs -ls /tmp

In [ ]:
hdfs dfs -cat /tmp/sparkSQLOut/part-00000

Схема

In [ ]:
dfArgdiv.printSchema()

Логические и физический планы выполнения

In [ ]:
dfArgdiv.explain(True)

In [ ]:
Отсортировать

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Обработка данных с Hive</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "3a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Запуск приложения на Cloudera
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#3b">Далее</a>
            </div>
        </div>
    </div>
</div>

<p>Отобразите версию Hive в Cloudera</p>

In [ ]:
hive --version

<p>Перейдите в интерактивный режим</p>

In [ ]:
hive

<div class="msg-block msg-warning">
  <span class="msg-text-warn">Предупреждение! </span>
  <p>Если не запускается, то попробуйте команду для выхода HDFS из безопасного режима</p>
      <div class="code-block"><p class="code-font"><span class="code-key">hdfs</span> dfsadmin -safemode leave</p></div>
</div>

<p>Удалите таблицу с именем divs, если она существует</p>

In [ ]:
DROP TABLE IF EXISTS divs;

<p>Вариант 1. Cоздайте стандартную таблицу</p>

In [ ]:
#Создаем таблицу
CREATE TABLE divs (exch STRING, symbol STRING, date DATE, dividends FLOAT) 
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' STORED AS TEXTFILE;
#Подгружаем данные
LOAD DATA INPATH '/tmp/NYSE_dividends.txt' OVERWRITE INTO TABLE divs;

In [ ]:
#Проверяем, что данные добавились в таблицу в HDFS 
hdfs dfs -ls /user/hive/warehouse/divs

<p>Вариант 2. Cоздайте внешнюю таблицу и укажите путь в HDFS, где она будет храниться данные. В этом случае при удалении таблицы данные сохранятся в указанной директории</p>

In [ ]:
CREATE EXTERNAL TABLE divs (exch STRING, symbol STRING, date DATE, dividends FLOAT) 
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' STORED AS TEXTFILE LOCATION '/tmp/hiveInput';

<div class="msg-block msg-warning">
  <span class="msg-text-warn">Предупреждение! </span>
  <p>Данные должны находиться в HDFS. Например, можно воспользоваться командой </p>
      <div class="code-block"><p class="code-font"><span class="code-key">hdfs</span> dfs -copyFromLocal /home/cloudera/NYSE_dividends.txt /tmp/hiveInput/NYSE_dividends.txt</p></div>
</div>

<p>Отобразите список таблиц</p>

In [ ]:
SHOW TABLES;

<p>Отобразите схему таблицы</p>

In [ ]:
DESCRIBE divs;

<p>Для удаления таблицы используется следующая команда</p>

In [ ]:
DROP TABLE divs;

<p>Отобразите первые пять строк</p>

In [ ]:
SELECT * FROM divs LIMIT 5;

<p>Определите для каждой группы среднее значение дивидентов</p>

In [ ]:
SELECT symbol, AVG(dividends) FROM divs GROUP BY symbol;

<p>Сохраните данные <span class="code-font"></span> в <span class="code-font code-key">HDFS</span></p>

In [ ]:
INSERT OVERWRITE DIRECTORY '/tmp/hiveOutput'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
SELECT symbol, AVG(dividends) FROM divs GROUP BY symbol;

<p>Проверьте, что файлы успешно созданы</p>

In [ ]:
hdfs dfs -ls /tmp/hiveOutput

<p>Проверьте содержание файлов</p>

In [ ]:
hdfs dfs -cat /tmp/hiveOutput/00*

<p>Отобразите планы выполнения</p>

In [ ]:
EXPLAIN SELECT symbol, AVG(dividends) FROM divs GROUP BY symbol;

In [ ]:
EXPLAIN DEPENDENCY SELECT symbol, AVG(dividends) FROM divs GROUP BY symbol;

<p>Для выхода из интерактивного режима</p>

In [ ]:
exit;

<p>Запустите файл с кодом</p>

In [ ]:
#Полный путь к файлу, в котором будет храниться код программы
hive_file = "YOUR_PATH/myHive.hql"

<p>Записываем код в файл <span class="code-font">myHive.hql</span></p>

In [ ]:
%%writefile $hive_file

DROP TABLE IF EXISTS divs;

CREATE EXTERNAL TABLE divs (exch STRING, symbol STRING, date DATE, dividends FLOAT) 
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' STORED AS TEXTFILE LOCATION '/tmp/hiveInput';

INSERT OVERWRITE DIRECTORY '/tmp/hiveOutput'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
SELECT symbol, AVG(dividends) FROM divs GROUP BY symbol;

DROP TABLE divs;

<p>Запустите скрипт</p>

hive -f YOUR_PATH/myHive.hql

<p>Проверьте результат выполнения</p>

In [ ]:
hdfs dfs -cat /tmp/hiveOutput/00*

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4 Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
PIG
http://chimera.labs.oreilly.com/books/1234000001811/index.html
http://spark.apache.org/docs/latest/sql-programming-guide.html
Spark
https://www.codementor.io/spark/tutorial/python-spark-sql-dataframes
http://spark.apache.org/docs/latest/programming-guide.html
Hive
https://cwiki.apache.org/confluence/display/Hive/GettingStarted
https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-ExternalTables
https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Cli
https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Types
https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Explain